![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

# Computational Literary Stylistics with R

This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Computational Literary Stylistics with R*](https://ladal.edu.au/litsty.html). 


***

**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial.


In [ ]:
# set options
options(stringsAsFactors = F)          # no automatic data transformation
options("scipen" = 100, "digits" = 12) # suppress math annotation
library(tidyverse)
library(janeaustenr)
library(tidytext)
library(forcats)
library(quanteda)
library(gutenbergr)


Once you have initiated the session by executing the code shown above, you are good to go.

If you are using this notebook on your own computer and you have not already installed the R packages listed above, you need to install them. You can install them by replacing the `library` command with `install.packages` and putting the name of the package into quotation marks like this: `install.packages("quanteda")`. Then, you simply run this command and R will install the package you specified.


# Getting started

To explore different methods used in literary stylistics, we will analyze selected works from the Project Gutenberg. For this tutorial, we will download William Shakespeare's *Romeo and Juliet*, Charles Darwin's *On the Origin of Species*, Edgar Allan Poe's *The Raven*, Jane Austen's *Pride and Prejudice*, Athur Conan Doyle's *The Adventures of Sherlock Holmes*, and Mark Twain's *The Adventures of Tom Sawyer* (to see how to download data from Project Gutenberg, check out this [tutorial](https://slcladal.github.io/gutenberg.html)).


In [ ]:
gb <- gutenberg_works()
gb %>% 
  filter(stringr::str_detect(author, "Shakespeare"),
         stringr::str_detect(title, "Romeo"))


The code below downloads the data from a server that mirrors the content of Project Gutenberg (which is more stable than the Project itself).



In [ ]:
shakespeare <- gutenbergr::gutenberg_works(gutenberg_id == "1513") %>%
  gutenbergr::gutenberg_download(mirror = "http://mirrors.xmission.com/gutenberg/")
darwin <- gutenbergr::gutenberg_works(gutenberg_id == "1228") %>%
  gutenbergr::gutenberg_download(mirror = "http://mirrors.xmission.com/gutenberg/")
twain <- gutenbergr::gutenberg_works(gutenberg_id == "74") %>%
  gutenbergr::gutenberg_download(mirror = "http://mirrors.xmission.com/gutenberg/")
poe <- gutenbergr::gutenberg_works(gutenberg_id == "1065") %>%
  gutenbergr::gutenberg_download(mirror = "http://mirrors.xmission.com/gutenberg/")
austen <- gutenbergr::gutenberg_works(gutenberg_id == "1342") %>%
  gutenbergr::gutenberg_download(mirror = "http://mirrors.xmission.com/gutenberg/")
doyle <- gutenbergr::gutenberg_works(gutenberg_id == "1661") %>%
  gutenbergr::gutenberg_download(mirror = "http://mirrors.xmission.com/gutenberg/")
# inspect
str(shakespeare)


***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Binder Folder Symbol](https://slcladal.github.io/images/binderfolder.JPG)


Then, when the menu has unfolded, click on the smaller folder symbol (encircled in red in the picture below).

![Small Binder Folder Symbol](https://slcladal.github.io/images/upload2.png)


Now, you are in the main menu and can click on the 'MyData' folder.

![MyData Folder Symbol](https://slcladal.github.io/images/upload3.png)


Now, that you are in the MyData folder, you can click on the upload symbol.

![Binder Upload Symbol](https://slcladal.github.io/images/binderupload.JPG)

Select and upload the files you want to analyze (**IMPORTANT: here, we assume that you upload some form of text data - not tabular data! You can upload only txt and docx files!**). When you then execute the code chunk below, you will upload your own data and you can then use it in this notebook.


In [ ]:
myfiles <- list.files(here::here("MyData"), # path to the corpus data
                          # full paths - not just the names of the files
                          full.names = T) 
# load files
mytext <- sapply(myfiles, function(x){
  x <- scan(x, 
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T)
  x <- paste0(x, sep = " ", collapse = " ")
  x <- stringr::str_squish(x)
})
# inspect
str(mytext)


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***

## Extracting words

The most basic but also most common task is to extract instances of individual words and seeing how they are used in context. This is also called *concordancing*. When extracting words, they are typically displayed in context which is why their display is called a *keyword in context concordance* or kwic, for short.

The code below extracts the word *pride* from the novel *Pride and Prejudice* and displays the resulting instances of this keyword in a kwic.


In [ ]:
# extract text
austen_text <- austen %>%
  dplyr::summarise(text = paste0(text, collapse = " ")) %>%
  stringr::str_squish()
# give text a name
names(austen_text)  <- "Pride & Prejudice"
# extract instances of pride
pride <- quanteda::kwic(austen_text, "pride") %>%
  as.data.frame()
# inspect
head(pride, 15)


The kwic display could now be processed further or could be inspected to see how the keyword in question (*pride*) is used in this novel.

We can also inspect the use of phrases, for example *natural selection*, expand the context window size, and clean the output (as shown below).


In [ ]:
# extract text
darwin_text <- darwin %>%
  dplyr::summarise(text = paste0(text, collapse = " ")) %>%
  stringr::str_squish()
# generate kwics
ns <- quanteda::kwic(darwin_text, phrase("natural selection"), window = 10) %>%
  as.data.frame() %>%
  dplyr::select(-docname, -from, -to, -pattern)
# inspect
head(ns)


## Identifying Keywords

Another common task in literary stylistics is to extract terms that are particularly characteristic of a given text. The problem underlying the identification of keywords is to figure out the importance of words in each document. We can assign weights to words that are more characteristic for a text if these terms are used more frequently than expected in a given text. We can then show terms ordered by their relative weight. Using the `bind_tf_idf()` function from the *tidytext* package, we can extract the *term frequency - inverse document frequency*, or tf-idf, scores which represent these relative weights and we can also  report other parameters such as number of occurrences of that word, total number of words and term frequency.

Before we continue, we need to define certain terms of concepts that are related to literary stylistics and that we will use repeatedly in this tutorials and that we need to define so that the analysis shown below makes sense.

*Term Frequency* is the measure of importance of a word in a document or how frequently it appears in that document. However there are some words such as *the*,*is*, *of*, etc. which appear frequently even though they might not be important. An approach of using a list of stop-words and removing them before analysis can be useful but in case of some documents these words might be highly relevant. 

The *Inverse Document Frequency* decreases the weight for most used words and increases the weight for words that are not much used in a collection of documents. This together with the Term Frequency can be used to calculate a term's *tf-idf* (the multiplication of both the terms) which adjusts the frequency of the term based on how rarely it is used. Mathematically *idf* can be expressed as follows:

\begin{equation}
  idf_{(term)}=  ln (\frac{n_{documents}}{n_{documents\; containing\; term}})
\end{equation}

Before we calculate the *tf-idf*, we will collapse all the books into a single data frame though.


In [ ]:
books <- rbind(shakespeare, darwin, twain, poe, austen, doyle)
# add names to books 
books$book <- c(
  rep("Shakespeare", nrow(shakespeare)),
  rep("Darwin", nrow(darwin)),
  rep("Twain", nrow(twain)),
  rep("Poe", nrow(poe)),
  rep("Austen", nrow(austen)),
  rep("Doyle", nrow(doyle))
)
# clean data
books <- books %>%
  dplyr::filter(text != "") %>%
  dplyr::mutate(book = factor(book)) %>%
  dplyr::select(-gutenberg_id)
# inspect
head(books)


Now, we continue by calculating the *tf-idf* for each term in each of the books.



In [ ]:
book_words <- books %>%
  tidytext::unnest_tokens(word, text) %>%
  dplyr::count(book, word, sort = TRUE) %>% 
  dplyr::group_by(book) %>% 
  dplyr::mutate(total = sum(n))
book_tf_idf <- book_words %>%
  tidytext::bind_tf_idf(word, book, n)
# inspect
head(book_tf_idf)


From the above table it is evident that the extremely common words have a very low inverse document frequency and thus a low tf-idf score. The inverse document frequency will be a higher number for words that occur in fewer documents in the collection of novels.



In [ ]:
book_tf_idf %>%
  dplyr::select(-total) %>%
  dplyr::arrange(desc(tf_idf))
# inspect
head(book_tf_idf)


Next, we plot the 15 words with the highest tf-idf scores for each novel to show which words are particularly characteristic of each of the novels.



In [ ]:
book_tf_idf %>%
  dplyr::group_by(book) %>%
  slice_max(tf_idf, n = 15) %>%
  dplyr::ungroup() %>%
  ggplot(aes(tf_idf, fct_reorder(word, tf_idf), fill = book)) +
  geom_col(show.legend = FALSE) +
  facet_wrap(~book, ncol = 2, scales = "free") +
  labs(x = "tf-idf", y = NULL)


As you can see, the method has indeed extracted words (and by extension concepts) that are characteristic of the texts. 

## Extracting Structural Features

Extracting structural features of texts is a very common and has a wide range of applications such as determining if texts belong to the same genre or if texts represent a real language or a made up nonsensical language, for example.

## Word-Frequency Distributions

Word-frequency distributions can be used to determine if a text represents natural language (or a simple replacement cipher, for example) or if the text does not represent natural language (or a more complex cipher). In the following, we will check if the language used in texts we have downloaded from Project Gutenberg aligns with distributions that we would expect when dealing with natural language. In a first step, we determine both the term-frequency and the idf.


In [ ]:
book_words <- books %>%
  tidytext::unnest_tokens(word, text) %>%
  dplyr::count(book, word, sort = TRUE)  %>% 
  dplyr::group_by(book) %>% 
  dplyr::mutate(total = sum(n))
# inspect
head(book_words)


From the above table it is evident that the usual suspects *the*, *and*, *to* and so-forth are leading in terms of their usage frequencies in the novels. Now let us look at the distribution of *n/total* for each term in each of the novels (which represents the normalized term frequency).



In [ ]:
ggplot(book_words, aes(n/total, fill = book)) +
  geom_histogram(show.legend = FALSE) +
  xlim(NA, 0.005) +
  facet_wrap(~book, ncol = 2, scales = "free_y")


From the plots it is clear that we are dealing with a negative exponential distribution and that many words occur only rarely and that only few words occur frequently. In other words, only few words occur frequently while most words occur rarely. This relationship represents a distribution that is captured by *Zipf's law*.


## Zipf's Law

Zipf's Law represents an empirical power law or power function that was established in the 1930s. Zipf's law is one of the most fundamental laws in linguistics [see @george1935zipf] and it states that the frequency of a word is inversely proportional to its rank in a text or collection of texts. 

Let

* N be the number of elements in a text (or collection of texts);

* k be their rank;

* s be the value of the exponent characterizing the distribution.

Zipf's law then predicts that out of a population of N elements, the normalized frequency of the element of rank k, f(k;s,N), is:

\begin{equation}
f(k;s,N)={\frac {1/k^{s}}{\sum \limits _{n=1}^{N}(1/n^{s})}} 
\end{equation}

In the code chunk below, we check if Zipf's Law applies to the words that occur in texts that we have downloaded from Project Gutenberg.


In [ ]:
freq_by_rank <- book_words %>% 
  dplyr::group_by(book) %>% 
  dplyr::mutate(rank = row_number(), 
         `term frequency` = n/total) %>%
  dplyr::ungroup()
# inspect
head(freq_by_rank)


To get a better understanding of Zipf's law, let us visualize the distribution by plotting on the logged rank of elements on the x-axis and logged frequency of the terms on the y-axis. If Zipf's law holds, then we should see more or less straight lines that go from top left to bottom right.



In [ ]:
freq_by_rank %>% 
  ggplot(aes(rank, `term frequency`, color = book)) + 
  geom_line(size = 1.1, alpha = 0.8, show.legend = FALSE) + 
  scale_x_log10() +
  scale_y_log10()


We can see that the plot has a negative slope which corroborates the inverse relationship of rank with respect to term frequency which shows that the words in the texts from Project Gutenberg follow Zipf's law. This would ascertain that we are dealing with natural language and not a made up nonsense language or a complex cipher. 

## Lexical Diversity

Lexical diversity is a complexity measure that provides information about the lexicon size of a text, i.e. how many different words occur in a text given the size of the text. The Type-Token-Ratio (TTR) provides information about the number of word tokens (individual instances of a word) divided by the number of different word types (word forms). Let's briefly elaborate on that and look a bit more closely at the terms *types* and *tokens*. The sentence *The dog chased the cat* contains five tokens but only 4 types because *the* occurs twice. Now, a text that is 100 words long and consist of 50 distinct words would have a TTR of .5 (50/100) while a text that is 100 words long but consist of 80 distinct words would have a TTR of .8 (80/100). Thus, typically, higher values indicate higher lexical diversity and more complex texts or more advanced learners of a language commonly have higher TTRs compared to simpler texts or less advanced language learners. 

As such, we can use lexical diversity measures to analyze the complexity of the language in which a text is written which can be used to inspect the advances a language learner makes when acquiring a language: initially, the learner will have high TTR as they do not have a large vocabulary. The TTRs will increase as lexicon of the learner grows.

In the following example, we calculate the TTRs for the literary texts we have downloaded from Project Gutenberg. Ina  first step, we tokenize the texts, i.e. we split the texts into individual words (tokens).


In [ ]:
books_texts <- books %>%
  dplyr::group_by(book) %>%
  dplyr::summarise(text = paste(text, collapse = " "))
texts <- books_texts %>%
  dplyr::pull(text)
names(texts) <- books_texts %>%
  dplyr::pull(book)
tokens_texts <- texts %>%
  quanteda::corpus() %>%
  quanteda::tokens()
# inspect data
head(tokens_texts)


Next, we calculate the TTR using the `textstat_lexdiv` function from the `quanteda` package and visualize the resulting TTRs for the literary texts that we have downloaded from Project Gutenberg.



In [ ]:
dfm(tokens_texts) %>%
  quanteda.textstats::textstat_lexdiv(measure = "TTR") %>%
  ggplot(aes(x = TTR, y = reorder(document, TTR))) + 
  geom_point() +
  xlab("Type-Token-Ratio (TTR)") +
  ylab("")


We can see that Darwin's *On the Origin of Species* has the lowest lexical diversity while Edgar Allan Poe's *The Raven* has the highest. This would suggest that the language in *The Raven* is more complex than the language of *On the Origin of Species*. However, this is too simplistic and shows that simple Type-Token Ratios are severely affected by text length (as well as orthographic errors) and should only be used to compare texts 

* that are comparatively long (at least 200 words) 
* that are approximately of the same length
* that were error corrected so that orthographic errors do not confound the ratios

### Average Sentence Length

The average sentence length (ASL) is another measure of textual complexity with more sophisticated language use being associated with longer and more complex sentences. As such, we can use the ASL as an alternative measure of the linguistic complexity of a text or texts. 


In [ ]:
library(lexRankr)
books_sentences <- books %>%
  dplyr::group_by(book) %>%
  dplyr::summarise(text = paste(text, collapse = " ")) %>%
  lexRankr::unnest_sentences(sentence, text)
# inspect
head(books_sentences)


Let's now visualize the results for potential differences or trends.



In [ ]:
books_sentences %>%
  dplyr::mutate(sentlength = stringr::str_count(sentence, '\\w+')) %>%
  ggplot(aes(x = sentlength, y = reorder(book, sentlength, mean), group = book)) +
  stat_summary(fun = mean, geom = "point")   +          
  stat_summary(fun.data = mean_cl_boot, geom = "errorbar", width = 0.2) +
  xlab("Average Sentence Length (ASL)") +
  ylab("")


We can see that that *The Raven* (which does not contain any punctuation) is (unsurprisingly) the text with the longest ASL while Shakespeare's play *Romeo and Juliet* (which contains a lost of dialogues) is deemed the work with the shortest ASL. With the exception of Poe's *The Raven*, the ALS results reflect text complexity with Darwin's *On the Origin of Species* being more complex or written like than the other texts with *Romeo and Juliet* being the most similar to spoken dialogue. 

## Similarity among literary texts

We will now explore how similar the language of literary works is. This approach can, of course, be extended to syntactic features or, for instance, to determine if certain texts belong to a certain literary genre or were written by a certain author. When extending this approach to syntactic features, one would naturally use features like the ALS, TTRs, or the frequency of adjectives as the basis for determining similarity. Regarding authorship, things like bigrams, spacing or punctuation methods would be relevant features. 

To assess the similarity of literary works (based on the words that occur in the texts), we first create a feature list, i.e. a matrix with word frequencies. In the present case, we remove stop words as well as symbols (it can be useful to retain these but this is depends on the task at hand).


In [ ]:
feature_mat <- books %>%
  dplyr::group_by(book) %>%
  dplyr::sample_n(100) %>%
  dplyr::summarise(text = paste0(text, collapse = " ")) %>%
  dplyr::ungroup() %>%
  quanteda::corpus(text_field = "text", docid_field = "book") %>%
  quanteda::dfm(remove_punct = TRUE, remove_symbols = TRUE) %>% 
  quanteda::dfm_remove(pattern = stopwords("en"))
# inspect data
feature_mat[1:6, 1:6]


We see that the texts are represented as the row names and the terms the column names. The content of the matrix consists of the term frequencies. 

We can now perform agglomerative hierarchical clustering and visualize the results in a dendrogram to assess the similarity of texts.


In [ ]:
books_dist <- as.dist(quanteda.textstats::textstat_dist(feature_mat))
books_clust <- hclust(books_dist)
plot(books_clust)


According to the dendrogram, Conan Doyle's *The Adventures of Sherlock Holmes* and Shakespeare's *Romeo and Juliet* are the most similar texts. Edgar Allen Poe's *The Raven* is the most idiosyncratic texts as it is on a branch by its own and is amalgamated with the other texts only as a very last step at the root of the tree.

## Networks of Personas

A final procedure we will perform is a network analysis of the personas in Shakespeare's *Romeo and Juliet*. We directly load a co-occurrence matrix which provides information about how often character's in that play have been in the same scene (as the extraction of this information is a bit cumbersome, I have done that for you and you can simply load the matrix into R). 


In [ ]:
# load data
romeo <- read.delim("https://slcladal.github.io/data/romeo.txt", sep = "\t")
# convert into feature co-occurrence matrix
romeo_fcm <- as.fcm(as.matrix(romeo))
# inspect data
romeo_fcm


As the `quanteda` package has a very neat and easy to use function (`textplot_network`) for generating network graphs, we make use this function and can directly generate the network.  



In [ ]:
quanteda.textplots::textplot_network(romeo_fcm, min_freq = 0.1, edge_alpha = 0.1, edge_size = 5)



The thickness of the lines indicates how often characters have co-occurred. We could now generate different network graphs for the personas in different plays to see how these plays and personas differ or we could apply the network analysis to other types of information such as co-occurrences of words.


We have reached the end of this tutorial. Please feel free to explore more of our content at https://ladal.edu.au - for computational literary stylistics, especially the tutorials on [part-of-speech tagging and syntactic parsing](https://ladal.edu.au/tagging.html) as well as on [lexicography with R](https://ladal.edu.au/lex.html) provide relevant additional information. 

***

[Back to LADAL](https://ladal.edu.au/kwics.html)

***
